In [1]:
import json
import os
import subprocess
import tempfile
import time
import traceback
from pprint import pprint

HOME_DIR = os.path.expanduser('~')
DEFAULT_LAKE_PATH = f'{HOME_DIR}/.elan/bin/lake'
DEFAULT_LEAN_WORKSPACE = 'mathlib4/'

LEAN4_DEFAULT_HEADER = "import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n"



In [2]:
class AttrDict(dict):
    """A dictionary that allows attribute-style access."""
    def __getattr__(self, name):
        return self[name]

    def __setattr__(self, name, value):
        self[name] = value

    def __delattr__(self, name):
        del self[name]

In [3]:
lake_path=DEFAULT_LAKE_PATH
lean_workspace=DEFAULT_LEAN_WORKSPACE
last_env=None
timeout=300
allTactics=True
ast=True
premises=True
tactics=True

In [4]:
from prover.lean.ast_parser import lean4_parser

In [5]:
def line_column_to_index(s: str, line: int, column: int) -> int:
    lines = s.split("\n")
    lines = [line + "\n" for line in lines]
    if line <= 0 or line > len(lines):
        raise ValueError("Line number out of range")
    if column < 0 or column >= len(lines[line - 1]):
        raise ValueError("Column number out of range")
    return sum(len(lines[i]) for i in range(line - 1)) + column

In [6]:
def verify_lean4_file(code,
    lake_path=DEFAULT_LAKE_PATH,
    lean_workspace=DEFAULT_LEAN_WORKSPACE,
    last_env=None,
    verbose=False,
    timeout=300,
    allTactics=False,
    ast=False,
    premises=False,
    tactics=False):
    command = dict(cmd=code, allTactics=allTactics, ast=ast,
                   tactics=tactics, premises=premises)
    if last_env is not None:
        command.update(env=last_env)
    message_str = json.dumps(command, ensure_ascii=False)
    if verbose:
        print(message_str)
    start_time = time.time()
    system_messages = ''
    try:
        with tempfile.TemporaryFile(mode='w+', encoding='utf-8') as temp_file:
            temp_file.write(message_str + "\r\n\r\n")
            temp_file.seek(0)
            outputs = subprocess.run([lake_path, "exe", 'repl'], stdin=temp_file,
                                     capture_output=True, text=True, cwd=lean_workspace, timeout=timeout)
        result = json.loads(outputs.stdout)
        ast_results = lean4_parser(
            code, result['ast']) if 'ast' in result and result['ast'] else {}
        result = {
            "sorries": result.get('sorries', []),
            "tactics": result.get('tactics', []),
            "errors": [m for m in result.get('messages', []) if m['severity'] == 'error'],
            "warnings": [m for m in result.get('messages', []) if m['severity'] == 'warning'],
            "infos": [m for m in result.get('messages', []) if m['severity'] == 'info'],
            "system_messages": system_messages,
            "system_errors": None,
            "ast": ast_results,
            "verified_code": code,
            "env": result.get('env', None),
        }
        result['pass'] = not result['errors']
        result['complete'] = result['pass'] and not result['sorries'] and not any(
            "declaration uses 'sorry'" in warning['data'] or 'failed' in warning['data'] for warning in result['warnings'])
    except:
        result = {
            "pass": False,
            "complete": False,
            "system_errors": traceback.format_exc(),
            "system_messages": system_messages
        }
    result['verify_time'] = time.time() - start_time
    return result

In [7]:

class Proof(object):
    def __init__(self,
                 code,
                 formal_statement,
                 header = LEAN4_DEFAULT_HEADER,
                 tailer = "",
                 ):
        
        self.code = code
        self.header = header
        self.formal_statement = formal_statement
        self.tailer = tailer
        self.full_code = ''.join([self.header, self.formal_statement, code.rstrip(' \n'), self.tailer])

        self.result = verify_lean4_file(self.full_code, ast = True, tactics = True)
        self._parse_full_code_lines()


    def _parse_full_code_lines(self):
        """
        Cache some basic string operations:
        the line offset of each line in the full code,
        and the full code lines.
        """
        self._full_code_lines = self.full_code.split('\n')
        self._line_offset, _offset = [], -1
        for _line in self._full_code_lines:
            _offset += 1  # '\n'
            self._line_offset.append(_offset)
            _offset += len(_line)

    def _get_idx(self, pos_info):
        """
        Convert a (line, column) dict to the index of the character in the full code.
        """
        return self._line_offset[pos_info['line'] - 1] + pos_info['column']

    def segmentation(self):
        result = self.result
        if 'errors' not in self.result:
            # compiler timeout
            return []
        
        """
        First, we need to find the last valid tactic.

        Unsolved goals also show up as errors, and we ignore them.
        """

        _prefix_len = len(self.header) + len(self.formal_statement)
        truncate_pos = len(self.full_code) - len(self.tailer)
        for info in result['sorries'] + result['errors']:
            if info.get('data', str()).lstrip().startswith('unsolved goals'):
                continue
            info_pos = self._get_idx(info['pos'])
            # if info_pos >= _prefix_len:
            truncate_pos = min(truncate_pos, info_pos)
        
        if truncate_pos <= _prefix_len:
            # all proof lines are invalid
            return []
        
        partial_code = self.full_code[:truncate_pos]

        code_lines = partial_code.split('\n')
        pos_last, segments = _prefix_len, []
        for line_idx in range(len(code_lines)):
            if self._line_offset[line_idx] >= _prefix_len:
                
                def compute_last_valid_char_pos(line):
                    idx, last_non_blank = 0, len(line) + 1
                    while idx < len(line):
                        if line[idx: idx+2] == '--':
                            return last_non_blank
                        elif line[idx: idx+2] == '/-':
                            if '-/' not in line[idx+2:]:
                                # cannot split in this line
                                return len(line) + 1
                            idx = line.find('-/', idx+2) + 1
                        elif line[idx] != ' ':
                            last_non_blank = idx
                        idx += 1
                    return last_non_blank
                

                line_lastChar = self._line_offset[line_idx] + \
                    compute_last_valid_char_pos(code_lines[line_idx])
                line_endPos = self._line_offset[line_idx] + \
                    len(code_lines[line_idx])

                pos_min, goal = 1e9, None
                for tactic_info in result['ast']['tactics']:
                    pos, endPos = tactic_info['pos'], tactic_info['endPos']
                    if line_lastChar <= endPos and endPos <= line_endPos and pos < pos_min:
                        pos_min = pos
                        goal = tactic_info['stateAfter']
                if goal is None:
                    continue
                
                for tactic_info in result['ast']['tactics']:
                    pos, endPos = tactic_info['pos'], tactic_info['endPos']
                    if pos_last < endPos and endPos <= line_endPos and pos < pos_min:
                        pos_min = pos

                while pos_min > 0 and partial_code[pos_min - 1] != '\n':
                    pos_min -= 1
                indent_len = 0
                while partial_code[pos_min + indent_len] == ' ':
                    indent_len += 1
                newline_with_indent = '\n' + ' ' * indent_len

                segments.append(AttrDict(
                    tactic_code=partial_code[pos_last: line_endPos] + '\n',
                    state_comment=newline_with_indent.join([
                        ' ' * indent_len + '/- tactic state:',
                        '  ' + goal.replace('\n',
                                            newline_with_indent + '  '),
                        '-/\n'
                    ]),
                    goal=goal,
                    indent=indent_len,
                ))
                pos_last = line_endPos + 1
        if result['complete'] and (len(segments) == 0 or segments[-1].goal != 'no goals' or segments[-1].indent != segments[0].indent):
            indent_len = 2 if len(segments) == 0 else segments[0].indent
            newline_with_indent = '\n' + ' ' * indent_len
            segments.append(AttrDict(
                tactic_code=partial_code[pos_last:].rstrip(' \n') + '\n',
                state_comment=newline_with_indent.join([
                    ' ' * indent_len + '/- tactic state:',
                    '  no goals',
                    '-/\n'
                ]),
                goal='no goals',
                indent=indent_len,
            ))
        segments = [seg for seg in segments if len(
            seg.tactic_code.strip(' \n')) > 0]
        return segments



In [8]:
problem = r"""/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
"""

code = r"""  -- First, we want to re-write the condition about the second
  -- and fourth terms of the geometric sequence using the definition of a geometric sequence
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  -- Now we can divide the two equations to eliminate $a$ and determine $r$
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  -- Finally, we can substitute back to find $a$
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄
"""

In [9]:
p = Proof(
    code = code,
    formal_statement=problem
)

In [14]:
print(p.__dict__)

{'code': "  -- First, we want to re-write the condition about the second\n  -- and fourth terms of the geometric sequence using the definition of a geometric sequence\n  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,\n    Nat.succ_add]\n  have h₁' : a * r = 2 := by simpa [h₀] using h₁\n  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂\n  -- Now we can divide the two equations to eliminate $a$ and determine $r$\n  have h₃ : r ^ 2 = 3 := by\n    nlinarith\n  -- Finally, we can substitute back to find $a$\n  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by\n    apply eq_or_eq_neg_of_sq_eq_sq <;>\n    field_simp <;>\n    nlinarith\n  simpa [h₀] using h₄\n", 'header': 'import Mathlib\nimport Aesop\n\nset_option maxHeartbeats 0\n\nopen BigOperators Real Nat Topology Rat\n\n', 'formal_statement': '/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?\nShow that it is $\\

In [12]:
segments = p.segmentation()
for i in segments:
    print("Tactic Code".center(80, '-'))
    print(i.tactic_code)
    print("State Comment".center(80, '-'))
    print(i.state_comment)

In [13]:
segments = p.segmentation()

In [18]:
old_code = r"""  -- First, we want to re-write the condition about the second
  -- and fourth terms of the geometric sequence using the definition of a geometric sequence
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  -- Now we can divide the two equations to eliminate $a$ and determine $r$
  have h₃ : r ^ 2 = 3 := by
"""

In [21]:
p.segmentation()
new_code = ""
i = 0
while len(new_code) <= len(old_code) and i < len(segments):
    new_code += segments[i].tactic_code
    i += 1

print(new_code)
assert new_code.startswith(old_code)

  -- First, we want to re-write the condition about the second
  -- and fourth terms of the geometric sequence using the definition of a geometric sequence
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  -- Now we can divide the two equations to eliminate $a$ and determine $r$
  have h₃ : r ^ 2 = 3 := by
    nlinarith



In [29]:
full_code = [
    r'''import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option linter.unusedVariables false
set_option linter.unnecessarySeqFocus false

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
''',
    r'''  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
''',
  r'''  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
''',
    r'''  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
''',
    r'''  simpa [h₀] using h₄
'''
]

In [30]:
print("".join(full_code))

import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option linter.unusedVariables false
set_option linter.unnecessarySeqFocus false

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄



In [18]:
# code = r"""
# import Mathlib
# import Aesop

# set_option maxHeartbeats 0
# set_option linter.unusedVariables false
# set_option linter.unnecessarySeqFocus false

# open BigOperators Real Nat Topology Rat

# /-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
# Show that it is $\frac{2\sqrt{3}}{3}$.-/
# theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
#   (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
#   simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
#     Nat.succ_add]
#   have h₁' : a * r = 2 := by simpa [h₀] using h₁
#   have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
#   have h₃ : r ^ 2 = 3 := by
#     nlinarith
# ><@>!@1248213
# sdbJKdsbkfj
#     apply eq_or_eq_neg_of_sq_eq_sq <;>
# """

In [19]:
code = r"""
import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option linter.unusedVariables false
set_option linter.unnecessarySeqFocus false

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
"""

In [20]:
results = verify_lean4_file(code, tactics=True, allTactics=True, premises = True)
for k, v in results.items():
    print(k.center(100, '-'))
    pprint(v)

----------------------------------------------sorries-----------------------------------------------
[]
----------------------------------------------tactics-----------------------------------------------
[{'endPos': {'column': 17, 'line': 16},
  'goals': 'a r : ℝ u : ℕ → ℝ h₀ : ∀ (k : ℕ), u k = a * r ^ k h₁ : u 1 = 2 h₂ '
           ': u 3 = 6 ⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
  'pos': {'column': 2, 'line': 15},
  'proofState': 0,
  'tactic': 'simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, '
            'Nat.add_succ, Nat.add_zero, Nat.succ_add]'},
 {'endPos': {'column': 48, 'line': 17},
  'goals': 'a r : ℝ\n'
           'u : ℕ → ℝ\n'
           'h₀ : ∀ (k : ℕ), u k = a * r ^ k\n'
           'h₁ : a * r ^ succ 0 = 2\n'
           'h₂ : a * r ^ 3 = 6\n'
           '⊢ a * r ^ 0 = 2 / √3 ∨ a * r ^ 0 = -(2 / √3)',
  'pos': {'column': 2, 'line': 17},
  'proofState': 1,
  'tactic': "have h₁' : a * r = 2 := by simpa [h₀] using h₁"},
 {'endPos': {'column': 48, 'line': 17},
  'go

In [196]:
for tactic in results['tactics']:
    print(tactic['pos'], tactic['endPos'])
    start = line_column_to_index(code, tactic['pos']['line'], tactic['pos']['column'])
    end = line_column_to_index(code, tactic['endPos']['line'], tactic['endPos']['column'])
    print(code[start:end])

{'line': 14, 'column': 2} {'line': 15, 'column': 17}
simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
{'line': 16, 'column': 2} {'line': 16, 'column': 48}
have h₁' : a * r = 2 := by simpa [h₀] using h₁
{'line': 16, 'column': 29} {'line': 16, 'column': 48}
simpa [h₀] using h₁
{'line': 17, 'column': 2} {'line': 17, 'column': 52}
have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
{'line': 17, 'column': 33} {'line': 17, 'column': 52}
simpa [h₀] using h₂
{'line': 18, 'column': 2} {'line': 19, 'column': 13}
have h₃ : r ^ 2 = 3 := by
    nlinarith
{'line': 19, 'column': 4} {'line': 19, 'column': 13}
nlinarith


In [202]:
def get_location_of_first_error(results):
    """
    Get the location of the first error in the results.
    """
    min_index = float('inf')
    first_error = None
    for error in results['errors']:
        start = line_column_to_index(code, error['pos']['line'], error['pos']['column'])
        if start < min_index:
            min_index = start
            first_error = error
    return min_index, first_error

In [204]:
print(code[:get_location_of_first_error(results)[0]])
print("---------ERROR BELOW---------")
print(get_location_of_first_error(results)[1])
print("---------ERROR ABOVE---------")
print(code[get_location_of_first_error(results)[0]:])


import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option linter.unusedVariables false
set_option linter.unnecessarySeqFocus false

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
>
---------ERROR BELOW---------
{'severity': 'error', 'pos': {'line': 21, 'column': 1}, 'endPos': {'line': 21, 'column': 2}, 'data': "unexpected token '<'; expected term"}
---------ERROR ABOVE---------
<@>!@1248213
sdbJKdsbkfj
    apply eq_

In [ ]:
def get_all_end_positions_from_tactics(results):
    """
    Get all end positions from tactics in the results.
    """
    return sorted(list(
        set(
            line_column_to_index(code, tactic['endPos']['line'], tactic['endPos']['column'])
        )
    ))

In [148]:
print("".join(code))

import Mathlib
import Aesop

set_option maxHeartbeats 0
set_option linter.unusedVariables false
set_option linter.unnecessarySeqFocus false

open BigOperators Real Nat Topology Rat

/-- The second and fourth terms of a geometric sequence are $2$ and $6$. Which of the following is a possible first term?
Show that it is $\frac{2\sqrt{3}}{3}$.-/
theorem amc12b_2003_p6 (a r : ℝ) (u : ℕ → ℝ) (h₀ : ∀ k, u k = a * r ^ k) (h₁ : u 1 = 2)
  (h₂ : u 3 = 6) : u 0 = 2 / Real.sqrt 3 ∨ u 0 = -(2 / Real.sqrt 3) := by
  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, Nat.add_succ, Nat.add_zero,
    Nat.succ_add]
  have h₁' : a * r = 2 := by simpa [h₀] using h₁
  have h₂' : a * r ^ 3 = 6 := by simpa [h₀] using h₂
  have h₃ : r ^ 2 = 3 := by
    nlinarith
  have h₄ : a = 2 / Real.sqrt 3 ∨ a = -(2 / Real.sqrt 3) := by
    apply eq_or_eq_neg_of_sq_eq_sq <;>
    field_simp <;>
    nlinarith
  simpa [h₀] using h₄



In [149]:
# pprint
from pprint import pprint

In [150]:
res = verify_lean4_file(
    code[0],
    # allTactics=True,
    tactics=True,
)
for k, v in res.items():
    print(k.center(100, '-'))
    pprint(v)

----------------------------------------------sorries-----------------------------------------------
[]
----------------------------------------------tactics-----------------------------------------------
[]
-----------------------------------------------errors-----------------------------------------------
[{'data': "unexpected end of input; expected '{'",
  'endPos': None,
  'pos': {'column': 0, 'line': 14},
  'severity': 'error'},
 {'data': 'unsolved goals\n'
          'a r : ℝ\n'
          'u : ℕ → ℝ\n'
          'h₀ : ∀ (k : ℕ), u k = a * r ^ k\n'
          'h₁ : u 1 = 2\n'
          'h₂ : u 3 = 6\n'
          '⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
  'endPos': {'column': 73, 'line': 13},
  'pos': {'column': 71, 'line': 13},
  'severity': 'error'}]
----------------------------------------------warnings----------------------------------------------
[]
-----------------------------------------------infos------------------------------------------------
[]
--------------------------------

In [128]:
res = verify_lean4_file(
    code[1],
    # allTactics=True,
    tactics=True,
    last_env = 0
)
for k, v in res.items():
    print(k.center(100, '-'))
    pprint(v)

----------------------------------------------sorries-----------------------------------------------
[]
----------------------------------------------tactics-----------------------------------------------
[]
-----------------------------------------------errors-----------------------------------------------
[]
----------------------------------------------warnings----------------------------------------------
[]
-----------------------------------------------infos------------------------------------------------
[]
------------------------------------------system_messages-------------------------------------------
''
-------------------------------------------system_errors--------------------------------------------
None
------------------------------------------------ast-------------------------------------------------
{}
-------------------------------------------verified_code--------------------------------------------
('  simp_all only [Nat.one_eq_succ_zero, Nat.zero_eq, zero_add, N

In [129]:
code[0].split("\n")[13]

''

In [132]:
p = subprocess.Popen(
    [lake_path, "exe", 'repl'],
    stdin=subprocess.PIPE,
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True,
    cwd=lean_workspace
)

In [133]:
p.stdin

<_io.TextIOWrapper name=76 encoding='UTF-8'>

In [134]:
command = dict(cmd=code[0], allTactics=True, ast=False,
                tactics=True, premises=False)
message_str = json.dumps(command, ensure_ascii=False)  + "\r\n\r\n"
p.stdin.write(message_str)


607

In [135]:
# check if the process is still running
if p.poll() is None:
    print("Process is still running")
else:
    print("Process has finished running")

Process is still running


In [136]:
p.communicate()

('{"messages":\n [{"severity": "error",\n   "pos": {"line": 14, "column": 0},\n   "endPos": null,\n   "data": "unexpected end of input; expected \'{\'"},\n  {"severity": "error",\n   "pos": {"line": 13, "column": 71},\n   "endPos": {"line": 13, "column": 73},\n   "data":\n   "unsolved goals\\na r : ℝ\\nu : ℕ → ℝ\\nh₀ : ∀ (k : ℕ), u k = a * r ^ k\\nh₁ : u 1 = 2\\nh₂ : u 3 = 6\\n⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)"}],\n "env": 0,\n "ast": []}\n\n',
 '')

In [137]:
# read the entire stdout of the process
p.stdout.readline()

ValueError: I/O operation on closed file.

In [88]:
for k, v in json.loads(stdout).items():
    print(k.center(100, '-'))
    pprint(v)

----------------------------------------------messages----------------------------------------------
[{'data': "unexpected end of input; expected '{'",
  'endPos': None,
  'pos': {'column': 0, 'line': 14},
  'severity': 'error'},
 {'data': 'unsolved goals\n'
          'a r : ℝ\n'
          'u : ℕ → ℝ\n'
          'h₀ : ∀ (k : ℕ), u k = a * r ^ k\n'
          'h₁ : u 1 = 2\n'
          'h₂ : u 3 = 6\n'
          '⊢ u 0 = 2 / √3 ∨ u 0 = -(2 / √3)',
  'endPos': {'column': 73, 'line': 13},
  'pos': {'column': 71, 'line': 13},
  'severity': 'error'}]
------------------------------------------------env-------------------------------------------------
0
------------------------------------------------ast-------------------------------------------------
[]


In [89]:
command = dict(cmd=code[1], allTactics=True, ast=False,
                tactics=True, premises=False, env=0)
message_str = json.dumps(command, ensure_ascii=False)  + "\r\n\r\n"
p.stdin.write(message_str)


ValueError: I/O operation on closed file.

In [76]:
stderr

''

In [107]:
s = r"""asdfhjasdhfvd
dwbjkfbajskdf
vsdajkvjlzxcv"""
get_index(s, 1, 3)

print(s[:get_index(s, 1, 3)])  # prints 'asdfhjasdhfvd\ndwbjkfba'

asdfhjasdhfvd
dwb
